In [17]:
chunk_size_base = 64

## Convert all netCDF NCAR timestep files to Zarr (64,64,64), round-robin across time across FileDB nodes

In [5]:
!pip install "xarray[complete]"

In [6]:
import xarray as xr
import os
# from dask.diagnostics import ProgressBar
# import zarr # No compute image with both xarray and zarr

<font color="orange">Don't delete the CD cell!</font>

In [7]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF

/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF


## Merge 3 velocity components into one 4D array

In [8]:
folder_path = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-netCDF/'
data_xr = xr.open_dataset(folder_path + "jhd.000.nc")
data_xr.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:  (nnx: 2048, nny: 2048, nnz: 2048)
Dimensions without coordinates: nnx, nny, nnz
Data variables:
    u        (nnz, nny, nnx) float32 ...
    v        (nnz, nny, nnx) float32 ...
    w        (nnz, nny, nnx) float32 ...
    t        (nnz, nny, nnx) float32 ...
    p        (nnz, nny, nnx) float32 ...
    e        (nnz, nny, nnx) float32 ...
Attributes:
    Simulation conducted by:  Peter Sullivan, pps@ucar.edu
    Dataset built by:         Edward Patton, patton@ucar.edu
    Affiliation:              NCAR/MMM
    Code:                     NCAR-LES
    Case Description:         GABLS 1, Cooling Rate: 0.25 K/hr
    Created:                  2023-05-02 10:25:55 -0600 UTC>

## Save Separate $512^3$

In [9]:
data_joint_vel_u = data_xr['u'][:512, :512, :512]
data_joint_vel_v = data_xr['v'][:512, :512, :512]
data_joint_vel_w = data_xr['w'][:512, :512, :512]

In [14]:
ds = xr.Dataset({'u': data_joint_vel_u, 'v': data_joint_vel_v, 'w': data_joint_vel_w})
ds.info

<bound method Dataset.info of <xarray.Dataset>
Dimensions:  (nnx: 512, nny: 512, nnz: 512)
Dimensions without coordinates: nnx, nny, nnz
Data variables:
    u        (nnz, nny, nnx) float32 ...
    v        (nnz, nny, nnx) float32 ...
    w        (nnz, nny, nnx) float32 ...>

In [16]:
ds['u']

<xarray.DataArray 'u' (nnz: 512, nny: 512, nnx: 512)>
[134217728 values with dtype=float32]
Dimensions without coordinates: nnz, nny, nnx
Attributes:
    Description:  Streamwise velocity, u
    Units:        [m/s]

In [21]:
encoding = {variable_name: {'compressor': None, 'chunks': (chunk_size_base,chunk_size_base,chunk_size_base)} for variable_name in ds.variables}
encoding

{'u': {'compressor': None, 'chunks': (64, 64, 64)},
 'v': {'compressor': None, 'chunks': (64, 64, 64)},
 'w': {'compressor': None, 'chunks': (64, 64, 64)}}

In [26]:
ds.to_zarr(store="/home/idies/workspace/turb/data01_01/zarr/joint_velocities_ariel_test_512.zarr",
           mode='w',
           encoding=encoding)

## Save Joint $512^3$

In [ ]:
# I don't think this will benefit from Parallelization
import dask.array as da


b = da.stack([data_xr['u'][:512, :512, :512], data_xr['v'][:512, :512, :512], data_xr['w'][:512, :512, :512]], axis=3)
# b = da.stack([data_xr['u'][:512, :512, :512], data_xr['v'][:512, :512, :512], data_xr['w'][:512, :512, :512]], axis=3)

# import numpy as np
# b = da.stack([np.zeros((128,128,128)), np.zeros((128,128,128)), np.zeros((128,128,128))], axis=3)

chunk_size_base = 64

# Need to edit chunk
b = b.rechunk((chunk_size_base,chunk_size_base,chunk_size_base,3))

b

In [9]:
da.to_zarr(b, url="/home/idies/workspace/turb/data02_02/zarr/" + "ariel_split_velocities_ariel_test_512.zarr", compressor=None)

In [12]:
# import zarr

# a = zarr.open("/home/idies/workspace/turb/data02_02/zarr/" + "ariel_split_velocities_ariel_test_512.zarr")

# a.info

In [ ]:
# I don't think this will benefit from Parallelization
import dask.array as da


b = da.stack([data_xr['u'], data_xr['v'], data_xr['w']], axis=3)

# import numpy as np
# b = da.stack([np.zeros((128,128,128)), np.zeros((128,128,128)), np.zeros((128,128,128))], axis=3)

chunk_size_base = 64

# Need to edit chunk
b = b.rechunk((chunk_size_base,chunk_size_base,chunk_size_base,3))

b

In [ ]:
# Remove individual velocity components, replace with joint

data_joint_vel = data_xr.drop_vars(['u', 'v', 'w'])

data_joint_vel.info

In [ ]:
# Add 3 velocities
data_joint_vel['velocity'] = xr.DataArray(b, dims=('nnz', 'nny', 'nnx', 'velocity component (xyz)'))

data_joint_vel.info

In [ ]:
data_joint_vel = data_joint_vel.drop_vars(['e', 'p', 't'])

In [ ]:
chunk_size_base = 64

# Disable compression, set chunk size
# encoding = {variable_name: {'compressor': None, 'chunks': (chunk_size_base, chunk_size_base, chunk_size_base)} for variable_name in data_joint_vel.variables}
encoding = {}
encoding['velocity'] = {'compressor': None, 'chunks': (chunk_size_base,chunk_size_base,chunk_size_base,3)}

encoding

In [ ]:
target_dir = folders[0]

# overwrite if exists
data_joint_vel.to_zarr(store="/home/idies/workspace/turb/data02_02/zarr/" + "ariel_split_velocities_ariel_test.zarr",
                mode="w",
               encoding = encoding)
